In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import nltk
import datetime

In [ ]:
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
train.head(5)

In [ ]:
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
test.shape

In [ ]:
train['date'] = pd.to_datetime(train['date'], format = '%d.%m.%Y')

In [ ]:
train['month'] = train['date'].dt.month

In [ ]:
train['year'] = train['date'].dt.year

In [ ]:
train = train.drop(['date', 'item_price'], axis = 1)

In [ ]:
train.head()

In [ ]:
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index = False)[['item_cnt_day']].sum()

In [ ]:
train = train.rename(columns = {'item_cnt_day': 'item_cnt_month'})

In [ ]:
train.head(5)

In [ ]:
shop_item_mean = train[['shop_id', 'item_id', 'item_cnt_month']].groupby(['shop_id', 'item_id'], as_index = False)[['item_cnt_month']].mean()

In [ ]:
shop_item_mean = shop_item_mean.rename(columns = {'item_cnt_month' : 'item_cnt_month_mean'})

In [ ]:
shop_item_mean

In [ ]:
train = pd.merge(train, shop_item_mean, how = 'left', on =['shop_id', 'item_id'])

In [ ]:
train.head()

In [ ]:
train.shape

adding last month

In [ ]:
shop_prev_month = train[train['date_block_num'] == 33][['shop_id', 'item_id', 'item_cnt_month']]

In [ ]:
shop_prev_month = shop_prev_month.rename(columns = {'item_cnt_month' : 'item_cnt_prev_month'})

In [ ]:
shop_prev_month.head()

In [ ]:
train = pd.merge(train, shop_prev_month, how = 'left', on=['shop_id', 'item_id']).fillna(0.)

In [ ]:
train = pd.merge(train, items, how = 'left', on = 'item_id')

In [ ]:
train = pd.merge(train, item_cats, how = 'left', on=['item_category_id'])

In [ ]:
train = pd.merge(train, shops, how ='left', on =['shop_id'])

In [ ]:
train.head()

**Test Dataset**

In [ ]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34

In [ ]:
test = pd.merge(test, shop_item_mean, how = 'left', on =['shop_id', 'item_id']).fillna(0.)

In [ ]:
test = pd.merge(test, shop_prev_month, how = 'left', on =['shop_id', 'item_id']).fillna(0.)

In [ ]:
test = pd.merge(test, items, how = 'left', on = 'item_id')

In [ ]:
test = pd.merge(test, item_cats, how = 'left', on = 'item_category_id')

In [ ]:
test = pd.merge(test, shops, how = 'left', on = 'shop_id')

In [ ]:
test.head()

In [ ]:
test['item_cnt_month'] = 0

**Label Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for c in ['shop_name', 'item_name', 'item_category_name']:
    lbl = LabelEncoder()
    lbl.fit(list(train[c].unique()) + list(test[c].unique()))
    train[c] = lbl.transform(train[c].astype(str))
    test[c] = lbl.transform(test[c].astype(str))

In [ ]:
col = [c for c in train.columns if c not in ['item_cnt_month']]
x1 = train[train['date_block_num'] < 33]
y1 = np.log1p(x1['item_cnt_month'].clip(0., 20.))
x1 = x1[col]
x2 = train[train['date_block_num'] == 33]
y2 = np.log1p(x2['item_cnt_month'].clip(0.,20.))
x2 = x2[col]


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
reg = ExtraTreesRegressor(n_estimators=25, n_jobs = -1, max_depth = 20, random_state=18)

In [ ]:
reg.fit(x1, y1)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print('RMSE value is : ', np.sqrt(mean_squared_error(y2.clip(0.,20.), reg.predict(x2).clip(0.,20.))))

In [ ]:
reg.fit(train[col], train['item_cnt_month'].clip(0.,20.))
test['item_cnt_month'] = reg.predict(test[col]).clip(0.,20.)
test[['ID', 'item_cnt_month']].to_csv('submission.csv', index = False)
test['item_cnt_month'] = np.expm1(test['item_cnt_month'])
test[['ID', 'item_cnt_month']].to_csv('final_submission.csv', index = False)